In [1]:
import tensorflow as tf
tf.config.experimental.set_visible_devices([], 'GPU')
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xlsxwriter

%matplotlib qt

# import module with layers
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/model_module")

from adaptive_layer import *

In [2]:
tf.random.set_seed(5)
np.random.seed(5)

In [3]:
def illustrate_train(name, y_label, static_f, adaptive_f, adaptive_nn, adaptive_conv_nn):
    epochs = range(1, len(static_f)+1)
    fig = plt.figure(figsize=(10,7))
    ax = fig.add_subplot()
    plt.title(name, fontsize="xx-large")

    ax.set_ylabel(y_label, fontsize="large")
    ax.set_xlabel('Epochs', fontsize="large")

    ax.plot(epochs, static_f, label='Tanh Function')
    ax.plot(epochs, adaptive_f, label='Swish Function')
    ax.plot(epochs, adaptive_nn, label='Adaptive NN')
    ax.plot(epochs, adaptive_conv_nn, label='Adaptive Conv NN')

    plt.xticks(range(1,len(epochs)+1))

    ax.legend()
    
    plt.grid()
    plt.show()

In [4]:
def illustrate_test(name, static_f, adaptive_f, adaptive_nn, adaptive_conv_nn):
    fig = plt.figure(figsize=(10,7))
    ax = fig.add_subplot()
    
    x = ["Tanh Function", "Swish Function", "Adaptive NN", "Adaptive Conv NN"]
    y = [static_f, adaptive_f, adaptive_nn, adaptive_conv_nn]
    
    plt.title(name, fontsize="xx-large")
    ax.set_ylabel("Accuracy", fontsize="large")
    
    ax.bar(x, y)
    
    plt.grid()
    plt.show()

In [5]:
workbook = xlsxwriter.Workbook('sbc.xlsx')

In [6]:
def save_data(name, static_f_train, adaptive_f_train, adaptive_nn_train, adaptive_conv_nn_train, 
              static_f_test, adaptive_f_test, adaptive_nn_test, adaptive_conv_nn_test):
    
    epochs = len(static_f_train[0][:])
    worksheet = workbook.add_worksheet(name)
    
    worksheet.write(1, 1, name + " Training Loss")
    worksheet.write(1, 7, name + " Training Accuracy")
    worksheet.write(1, 13, name + " Test Accuracy")
    
    worksheet.write(2, 1, "Epochs")
    worksheet.write(2, 2, "Tanh Function")
    worksheet.write(2, 3, "Swish Function")
    worksheet.write(2, 4, "Adaptive NN")
    worksheet.write(2, 5, "Adaptive Conv NN")
    
    worksheet.write(2, 7, "Epochs")
    worksheet.write(2, 8, "Tanh Function")
    worksheet.write(2, 9, "Swish Function")
    worksheet.write(2, 10, "Adaptive NN")
    worksheet.write(2, 11, "Adaptive Conv NN")
    
    worksheet.write(2, 13, "Static Function")
    worksheet.write(2, 14, "Adaptive Function")
    worksheet.write(2, 15, "Adaptive NN")
    worksheet.write(2, 16, "Adaptive Conv NN")
    
    worksheet.write(3, 13, static_f_test)
    worksheet.write(3, 14, adaptive_f_test)
    worksheet.write(3, 15, adaptive_nn_test)
    worksheet.write(3, 16, adaptive_conv_nn_test)
    
    for i in range(0, epochs):
        worksheet.write(3+i, 1, i+1)
        worksheet.write(3+i, 2, static_f_train[0][i])
        worksheet.write(3+i, 3, adaptive_f_train[0][i])
        worksheet.write(3+i, 4, adaptive_nn_train[0][i])
        worksheet.write(3+i, 5, adaptive_conv_nn_train[0][i])
        
        worksheet.write(3+i, 7, i+1)
        worksheet.write(3+i, 8, static_f_train[1][i])
        worksheet.write(3+i, 9, adaptive_f_train[1][i])
        worksheet.write(3+i, 10, adaptive_nn_train[1][i])
        worksheet.write(3+i, 11, adaptive_conv_nn_train[1][i])
    
    workbook.close()

In [7]:
# dataset

x_train = tf.random.uniform(shape=(2000, 1), minval = 0, maxval = 1)
y_train = np.array([a > 0.5 for a in (x_train)], dtype=float)

x_test = tf.random.uniform(shape=(500, 1), minval = 0, maxval = 1)
y_test = np.array([a > 0.5 for a in (x_test)], dtype=float)

In [8]:
EPOCHS = 5
name = "Number Binary Classification"

# measurement repetitions
flag = measurements = 5

static_f_hist = np.zeros(shape=[2, EPOCHS])
adaptive_f_hist = np.zeros(shape=[2, EPOCHS])
adaptive_nn_hist = np.zeros(shape=[2, EPOCHS])
adaptive_conv_nn_hist = np.zeros(shape=[2, EPOCHS])

static_f_test = 0
adaptive_f_test = 0
adaptive_nn_test = 0
adaptive_conv_nn_test = 0

In [95]:
static_f_model = keras.Sequential([
    keras.layers.Dense(2, activation="tanh"),
    keras.layers.Dense(1, activation="sigmoid")
])

In [96]:
adaptive_f_model = keras.Sequential([
    keras.layers.Dense(2, activation=tf.nn.swish),
    keras.layers.Dense(1, activation="sigmoid")
])

In [97]:
# Adaptive Layer
adaptive_nn_model = AdaptiveModel([
    AdaptiveLayer(2, structure=[2, 1], inner_out_activation=tf.nn.sigmoid),
    AdaptiveLayer(1, structure=[2, 1], inner_out_activation=tf.nn.sigmoid)
])

In [98]:
# Adaptive Layer based on 1D Conv
adaptive_conv_nn_model = AdaptiveModel([
    AdaptiveLayerConv(2, structure=[2, 4], skip_w=0, noise=0, inner_out_activation=tf.nn.sigmoid),
    AdaptiveLayerConv(1, structure=[2, 4], skip_w=0, noise=0, inner_out_activation=tf.nn.sigmoid)
])

In [99]:
static_f_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.01), 
                       loss="binary_crossentropy", metrics=['accuracy'])

In [100]:
adaptive_f_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.01), 
                       loss="binary_crossentropy", metrics=['accuracy'])

In [101]:
adaptive_nn_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.01), 
                       loss="binary_crossentropy", metrics=['accuracy'])

In [102]:
adaptive_conv_nn_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.01), 
                       loss="binary_crossentropy", metrics=['accuracy'])

In [103]:
hist = static_f_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)

Epoch 1/5
32/32 [==============================] - 0s 523us/step - loss: 0.5634 - accuracy: 0.7355
Epoch 2/5
32/32 [==============================] - 0s 493us/step - loss: 0.4436 - accuracy: 0.9235
Epoch 3/5
32/32 [==============================] - 0s 478us/step - loss: 0.3359 - accuracy: 0.9715
Epoch 4/5
32/32 [==============================] - 0s 445us/step - loss: 0.2587 - accuracy: 0.9675
Epoch 5/5
32/32 [==============================] - 0s 447us/step - loss: 0.2073 - accuracy: 0.9800


In [104]:
static_f_hist[0][:] += np.array(hist.history["loss"])/measurements
static_f_hist[1][:] += np.array(hist.history["accuracy"])/measurements

In [105]:
hist = adaptive_f_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)

Epoch 1/5
32/32 [==============================] - 0s 510us/step - loss: 0.6294 - accuracy: 0.7355
Epoch 2/5
32/32 [==============================] - 0s 503us/step - loss: 0.5554 - accuracy: 0.9375
Epoch 3/5
32/32 [==============================] - 0s 460us/step - loss: 0.4550 - accuracy: 0.9895
Epoch 4/5
32/32 [==============================] - 0s 434us/step - loss: 0.3568 - accuracy: 0.9855
Epoch 5/5
32/32 [==============================] - 0s 438us/step - loss: 0.2816 - accuracy: 0.9870


In [106]:
adaptive_f_hist[0][:] += np.array(hist.history["loss"])/measurements
adaptive_f_hist[1][:] += np.array(hist.history["accuracy"])/measurements

In [107]:
hist = adaptive_nn_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)

Epoch 1/5
32/32 [==============================] - 0s 556us/step - loss: 0.6933 - accuracy: 0.5150
Epoch 2/5
32/32 [==============================] - 0s 613us/step - loss: 0.6706 - accuracy: 0.6205
Epoch 3/5
32/32 [==============================] - 0s 532us/step - loss: 0.3839 - accuracy: 0.9675
Epoch 4/5
32/32 [==============================] - 0s 610us/step - loss: 0.1011 - accuracy: 0.9850
Epoch 5/5
32/32 [==============================] - 0s 535us/step - loss: 0.0504 - accuracy: 0.9955


In [108]:
adaptive_nn_hist[0][:] += np.array(hist.history["loss"])/measurements
adaptive_nn_hist[1][:] += np.array(hist.history["accuracy"])/measurements

In [109]:
hist = adaptive_conv_nn_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)

Epoch 1/5
32/32 [==============================] - 0s 751us/step - loss: 0.6942 - accuracy: 0.4920
Epoch 2/5
32/32 [==============================] - 0s 762us/step - loss: 0.6929 - accuracy: 0.5680
Epoch 3/5
32/32 [==============================] - 0s 806us/step - loss: 0.4496 - accuracy: 0.8805
Epoch 4/5
32/32 [==============================] - 0s 740us/step - loss: 0.0684 - accuracy: 0.9940
Epoch 5/5
32/32 [==============================] - 0s 752us/step - loss: 0.0277 - accuracy: 0.9920


In [110]:
adaptive_conv_nn_hist[0][:] += np.array(hist.history["loss"])/measurements
adaptive_conv_nn_hist[1][:] += np.array(hist.history["accuracy"])/measurements

In [111]:
static_f_test += static_f_model.evaluate(x_test, y_test)[1]/measurements

16/16 [==============================] - 0s 532us/step - loss: 0.1943 - accuracy: 0.9840


In [112]:
adaptive_f_test += adaptive_f_model.evaluate(x_test, y_test)[1]/measurements

16/16 [==============================] - 0s 460us/step - loss: 0.2525 - accuracy: 0.9900


In [113]:
adaptive_nn_test += adaptive_nn_model.evaluate(x_test, y_test)[1]/measurements

16/16 [==============================] - 0s 596us/step - loss: 0.0469 - accuracy: 0.9900


In [114]:
adaptive_conv_nn_test += adaptive_conv_nn_model.evaluate(x_test, y_test)[1]/measurements
flag -= 1
if(flag == 0):
    print("---------FINISH---------")

16/16 [==============================] - 0s 514us/step - loss: 0.0203 - accuracy: 0.9900
---------FINISH---------


In [115]:
print("Static Function model\nTest accuracy : ", static_f_test)
print("\nAdaptive Function model\nTest accuracy : ", adaptive_f_test)
print("\nAdaptive NN model\nTest accuracy : ", adaptive_nn_test)
print("\nAdaptive Conv NN model\nTest accuracy : ", adaptive_conv_nn_test)

Static Function model
Test accuracy :  0.9736000061035157

Adaptive Function model
Test accuracy :  0.9836000084877015

Adaptive NN model
Test accuracy :  0.9900000095367432

Adaptive Conv NN model
Test accuracy :  0.9896000027656555


In [116]:
illustrate_train(name+" Training Loss", "Loss", static_f_hist[0][:], 
                 adaptive_f_hist[0][:], adaptive_nn_hist[0][:], adaptive_conv_nn_hist[0][:])

In [117]:
illustrate_train(name + " Training Accuracy", "Accuracy", static_f_hist[1][:], 
                 adaptive_f_hist[1][:], adaptive_nn_hist[1][:], adaptive_conv_nn_hist[1][:])

In [118]:
illustrate_test(name + " Test Accuracy", 
                static_f_test, adaptive_f_test, adaptive_nn_test, adaptive_conv_nn_test)

In [119]:
save_data(name, static_f_hist, adaptive_f_hist, adaptive_nn_hist, 
          adaptive_conv_nn_hist, static_f_test, adaptive_f_test, adaptive_nn_test, adaptive_conv_nn_test)